In [59]:
import pandas as pd
import numpy as np  

In [60]:
data = pd.read_csv("d:\\DTD\\Project\\Dataset\\1000_Companies_Raw.csv")

In [61]:
data.info()
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002 entries, 0 to 1001
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        1001 non-null   float64
 1   Administration   1000 non-null   float64
 2   Marketing Spend  999 non-null    float64
 3   State            1002 non-null   object 
 4   Profit           998 non-null    object 
dtypes: float64(3), object(2)
memory usage: 39.3+ KB


(1002, 5)

In [62]:
data.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,NEW York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [63]:
data.describe()

,R&D Spend,Administration,Marketing Spend
count,1001.000000,1000.000000,999.000000
mean,81464.154046,122947.707545,225783.520802
std,46565.313865,12622.275483,91704.343044
min,0.000000,51283.140000,0.000000
25%,43002.000000,116625.436400,149696.595200
50%,79611.000000,122421.612150,223872.468700
75%,123926.000000,129139.118000,307365.910850
max,165349.200000,321652.140000,471784.100000


In [64]:
# Remove Duplicate Rows
before = len(data)
print(f"Number of rows before removing duplicates: {before}")

Number of rows before removing duplicates: 1002


In [65]:
data = data.drop_duplicates()
after = len(data)
print(f"\nRemoved {before - after} duplicate rows.")
print(f"Number of rows after removing duplicates: {after}")
data.shape


Removed 3 duplicate rows.
Number of rows after removing duplicates: 999


(999, 5)

In [66]:
#STRIP WHITESPACE FROM COLUMN NAMES
data.columns = data.columns.str.strip()

In [67]:
# Fixing the profit column
data["Profit"] = pd.to_numeric(data["Profit"], errors="coerce")


In [68]:
# Handling missing data
# Checking for missing values
data.isnull().sum()


R&D Spend          1
Administration     2
Marketing Spend    3
State              0
Profit             6
dtype: int64

In [69]:
# Columns to fill using median
cols_to_fill_median = ["Administration"]
for col in cols_to_fill_median:
    median_value = data[col].median()
    data[col].fillna(median_value, inplace=True)

#Columns to fill using mean
cols_to_fill_mean = ["R&D Spend", "Marketing Spend"]
for col in cols_to_fill_mean:
    mean_value = data[col].mean()
    data[col].fillna(mean_value, inplace=True)

# Deleting rows with missing values in 'Profit' column
data.dropna(subset=['Profit'], inplace=True)

# Final check for missing values
data.isnull().sum()

C:\Users\reshm\AppData\Local\Temp\ipykernel_8796\1808909511.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(median_value, inplace=True)
C:\Users\reshm\AppData\Local\Temp\ipykernel_8796\1808909511.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

In [70]:
# STANDARDIZE TEXT FORMATTING
#Fix known anomalies manually (if needed)
data['State'] = data['State'].replace({
    'Newyork': 'New York',
    'California': 'California',
    'Cali Fornia': 'California',
    'New York': 'New York',
    'Florida': 'Florida'
})

# Verify counts before and after
print("Before standardization:")
print(data['State'].value_counts(dropna=False))

# Fix inconsistent spelling/case for State
data["State"] = (data["State"].str.strip().str.title())
     # Converts NEW york → New York, CALIFORNIA → California
# spelling correction for california
data["State"] = data["State"].replace({"Cali Fornia": "California"})
print("\nAfter standardization:")
print(data['State'].value_counts(dropna=False))




Before standardization:
State
California     337
New York       321
Florida        321
NEW York        10
Cali fornia      4
Name: count, dtype: int64

After standardization:
State
California    341
New York      331
Florida       321
Name: count, dtype: int64


In [71]:
# Handling outliers for unrealistic values in numeric columns
numeric_cols = ["R&D Spend", "Marketing Spend"]
for col in numeric_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]



In [72]:
#Cleaned data shape
data.shape

(993, 5)

In [73]:
# Cleaned data preview
clean_path = "d:\\DTD\\Project\\cleaned_1000_companies.csv"
data.to_csv(clean_path, index=False)